In [54]:
import json
import os
from datetime import date

class ElasticModel:
    def __init__(self) -> None:
        self.患者名= ""
        self.住院号 = 0
        self.入院时间= date.today().strftime('%Y-%m-%d')
        self.出院时间= date.today().strftime('%Y-%m-%d')
        self.文件类型=''
        self.文件存储目录=''
        self.文件名称=''
        self.页号=0
        self.页内容=''

    def parse_fname(self,path,fname):
        txt_list = fname.split("_")
        print(txt_list)
        self.患者名=""
        self.住院号=txt_list[0]
        self.文件类型=txt_list[1]
        self.文件存储目录=path +"/" +fname
        self.文件名称=fname

    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, indent=4,ensure_ascii=False)
    
esmodel = ElasticModel()
esmodel.parse_fname('C:/temp/501372585','501372585_出院记录_出院记录_0001UD10000001AIOG7W_1.pdf')

print(esmodel.toJSON())

['501372585', '出院记录', '出院记录', '0001UD10000001AIOG7W', '1.pdf']
{
    "患者名": "",
    "住院号": "501372585",
    "入院时间": "2023-07-23",
    "出院时间": "2023-07-23",
    "文件类型": "出院记录",
    "文件存储目录": "C:/temp/501372585/501372585_出院记录_出院记录_0001UD10000001AIOG7W_1.pdf",
    "文件名称": "501372585_出院记录_出院记录_0001UD10000001AIOG7W_1.pdf",
    "页号": 0,
    "页内容": ""
}


In [72]:
import fitz # pymupdfライブラリ
class ProcessPDF:
    def __init__(self,es,dbname) -> None:
        self.es = es
        self.dbname = dbname

    def pdf_to_search(self,path,fname):
        esmodel = ElasticModel()
        print(path)
        print(fname)
        esmodel.parse_fname(path,fname)
        path = path +"/" +fname
        print(path)
        doc = fitz.open(path)
        for page in range(len(doc)):
            text = doc[page].get_text()
            text = text.replace('\n', '')
            esmodel.页号 = page
            esmodel.页内容 = text
            print(esmodel)
            self.es.index(index=self.dbname, body=esmodel.toJSON())

    def process(self,dir):
        listFiles = os.listdir(dir)
        print(listFiles)
        for file in listFiles :
            self.pdf_to_search(dir,file)


In [73]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
db = 'medical_records'  # 索引名称
pp = ProcessPDF(es,db)
pp.process("C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585")

['501372585_体温单_体温单_1.pdf', '501372585_体温单_体温单_2.pdf', '501372585_体温单_体温单_3.pdf', '501372585_入院记录_神内五入院记录_0001UD100000019ZAWKO_1.pdf', '501372585_入院证_入院证_3240c2aa-57e5-4508-ad94-7f9c10b37326_1.pdf', '501372585_其他记录_风险评估单_0001UD100000019ZAWL0_1.pdf', '501372585_出院记录_出院记录_0001UD10000001AIOG7W_1.pdf', '501372585_医嘱_临时_神经内科五病区ZB12020205_1.pdf', '501372585_医嘱_长期_神经内科五病区ZB12020205_1.pdf', '501372585_护理_住院患者自理能力评估单_0001UD100000019ZAWEN_1.pdf', '501372585_护理_入院患者护理评估单_0001UD100000019ZAWDR_1.pdf', '501372585_护理_普通病区住院患者须知_0001UD100000019ZAWDS_1.pdf', '501372585_护理_普通病区跌倒、坠床危险因素评估单_0001UD100000019ZAWEA_1.pdf', '501372585_护理_疼痛评估护理记录单_0001UD100000019ZBHO2_1.pdf', '501372585_护理_神经内科疾病患者健康宣教单_0001UD100000019ZAWDQ_1.pdf', '501372585_护理_血糖检测护理记录单_1_1.pdf', '501372585_检查_常规心电图检查(十八导联)_神经内科五病区b7b02917-bfe7-4e6b-bec3-e5ae974fb58b.pdf', '501372585_检验_BNP_ZB12020205神经内科五病区8377009_1.pdf', '501372585_检验_GLU,脂四_ZB12020205神经内科五病区8377021_1.pdf', '501372585_检验_二磷酸腺苷诱导血小板聚集_ZB12020205神经内科五病区8378302_1.pdf', '5013

C:\Users\john wang\AppData\Local\Temp\ipykernel_26832\3243221693.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  self.es.index(index=self.dbname, body=esmodel.toJSON())


C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585
501372585_体温单_体温单_2.pdf
['501372585', '体温单', '体温单', '2.pdf']
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585/501372585_体温单_体温单_2.pdf
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585
501372585_体温单_体温单_3.pdf
['501372585', '体温单', '体温单', '3.pdf']
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585/501372585_体温单_体温单_3.pdf
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585
501372585_入院记录_神内五入院记录_0001UD100000019ZAWKO_1.pdf
['501372585', '入院记录', '神内五入院记录', '0001UD100000019ZAWKO', '1.pdf']
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585/501372585_入院记录_神内五入院记录_0001UD100000019ZAWKO_1.pdf
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/501372585
501372585_入院证_入院证_3240c2aa-57e5-4508-ad94-7f9c10b37326_1.pdf
['501372585', '入院证', '入院证', '3240c2aa-57e5-4508-ad94-7f9c10b37326', '1.pdf']
C:/Users/john wang/Desktop/工作/007-project/elasticsearch/50137258